# 検索ランキングをDNNで回帰分析して予測する

### 大まかな処理の流れ
1. データの取り込み
2. データの整形（トレーニングデータとテストデータを作成）
3. モデルの定義
4. チューナーをインスタンス化して、ハイパーパラメータのチューニングする
5. チューニングしたハイパーパラメータでモデルを作成し、トレーニングする
6. 作成したモデルを保存する
7. 保存したモデルをお読み込み、予測したいデータで予測する

## 使用ライブラリのインポート

In [1]:
import tensorflow as tf
import pandas as pd
from keras.utils import FeatureSpace
import json

## 1. データの取り込み

In [2]:
# 予測に使用したいデータに応じて変更する
with open('ranking_data.json') as f:
    json_data = json.load(f)

dataframe = pd.read_json(json.dumps(json_data['data']))

In [3]:
# ランキングをカテゴリー分け
dataframe['rank'].mask(dataframe['rank'] == 1, 0, inplace=True)
dataframe['rank'].mask((dataframe['rank'] == 2) & (dataframe['rank'] == 3), 1, inplace=True)
dataframe['rank'].mask((4 <= dataframe['rank']) & (dataframe['rank'] <= 10), 2, inplace=True)
dataframe['rank'].mask(dataframe['rank'] > 10, 3, inplace=True)

## 2. データの確認
省略可能

In [4]:
print(dataframe.shape)

(1000, 22)


In [5]:
dataframe.head()

,bankLink,waitTime,returnRate,ppu,totalImgByte,imageNum,newVisitRate,mainContentHeight,imgLoadTime,jsResponseTime,...,cssNum,jsByte,jsNum,keywordNum,headLineNum,isHeadKLine1Keyword,titleNum,isTitleKeyword,isSiteIndex,rank
0,1221,192,76,20,3498621,84,78,10994,8.407202,517,...,14,3979530,31,13,43,1,69,0,0,3
1,1119,179,79,10,1593458,92,61,5774,3.001431,995,...,11,3574088,81,18,24,0,60,0,1,2
2,300,31,38,16,2398586,44,57,2957,3.163140,817,...,7,2606387,97,19,43,1,70,1,1,0
3,1276,297,41,18,3825689,10,53,13604,9.909420,313,...,12,2188462,84,5,16,0,124,1,0,3
4,477,113,87,27,6021409,122,40,17770,5.565168,904,...,19,2295419,49,4,26,0,141,0,1,2


## 3. トレーニングデータと検証データの作成

In [6]:
# データフレームから検証用としてランダムに20％取り出す
# random_stateを固定しているので、実行ごとに検証データとして取り出すデータは同じ
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 800 samples for training and 200 for validation


In [7]:
# 予測に使用したいデータに応じて変更する
predict_value = "rank"

# データフレームからデータセットに変換
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = tf.keras.utils.to_categorical(dataframe.pop(predict_value), num_classes=4)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [8]:
# データセットの中身の確認
for x, y in train_ds.take(1):
    print("Input:", x)
    print(f"{predict_value}:", y)

Input: {'bankLink': <tf.Tensor: shape=(), dtype=int64, numpy=1403>, 'waitTime': <tf.Tensor: shape=(), dtype=int64, numpy=218>, 'returnRate': <tf.Tensor: shape=(), dtype=int64, numpy=51>, 'ppu': <tf.Tensor: shape=(), dtype=int64, numpy=11>, 'totalImgByte': <tf.Tensor: shape=(), dtype=int64, numpy=2915930>, 'imageNum': <tf.Tensor: shape=(), dtype=int64, numpy=66>, 'newVisitRate': <tf.Tensor: shape=(), dtype=int64, numpy=51>, 'mainContentHeight': <tf.Tensor: shape=(), dtype=int64, numpy=17781>, 'imgLoadTime': <tf.Tensor: shape=(), dtype=float64, numpy=8.280564189305464>, 'jsResponseTime': <tf.Tensor: shape=(), dtype=int64, numpy=383>, 'htmlTotalByte': <tf.Tensor: shape=(), dtype=int64, numpy=23156>, 'cssByte': <tf.Tensor: shape=(), dtype=int64, numpy=116108>, 'cssNum': <tf.Tensor: shape=(), dtype=int64, numpy=11>, 'jsByte': <tf.Tensor: shape=(), dtype=int64, numpy=2420833>, 'jsNum': <tf.Tensor: shape=(), dtype=int64, numpy=83>, 'keywordNum': <tf.Tensor: shape=(), dtype=int64, numpy=15>, '

In [9]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [10]:
# 予測に使用したいデータに応じて変更する
# 構造化データの前処理とエンコードを行う
# 各データの特徴を以下の分類に当てはめる。
# "integer_categorical"：数値（0,1など）カテゴリー、"string_categorical"：文字列カテゴリー、"float_discretized"：離散化する数値的特徴、"float_normalized"：正規化する数値的特徴
feature_space = FeatureSpace(
    features={
        # 数値カテゴリー特徴
        "isHeadKLine1Keyword": "integer_categorical",
        "isTitleKeyword": "integer_categorical",
        "isSiteIndex": "integer_categorical",
        # 正規化する数値的特徴
        "bankLink": "float_normalized",
        "waitTime": "float_normalized",
        "returnRate": "float_normalized",
        "ppu": "float_normalized",
        "totalImgByte": "float_normalized",
        "imageNum": "float_normalized",
        "newVisitRate": "float_normalized",
        "mainContentHeight": "float_normalized",
        "imgLoadTime": "float_normalized",
        "jsResponseTime": "float_normalized",
        "htmlTotalByte": "float_normalized",
        "cssByte": "float_normalized",
        "cssNum": "float_normalized",
        "jsByte": "float_normalized",
        "jsNum": "float_normalized",
        "keywordNum": "float_normalized",
        "headLineNum": "float_normalized",
        "titleNum": "float_normalized",
    },
    # "concat"または"dict"。
    # "concat"では、すべての特徴が 1 つのベクトルに連結される。
    # "dict"では、個別にエンコードされた特徴のdictを返す (入力キーと同じキーを使用)。
    output_mode="concat",
)

In [11]:
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)
feature_space.save("rankingcategoryfeaturespace.keras")

In [12]:
# トレーニングデータセットから１つ取り出して確認
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)

preprocessed_x.shape: (32, 27)
preprocessed_x.dtype: <dtype: 'float32'>


In [13]:
# num_parallel_callsで処理を並列化する。
# tf.data.AUTOTUNEは並列度をランタイムで良い感じに決めてくれる。
preprocessed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
# prefetchはGPUが計算している間にBatchデータをCPU側で用意しておく機能
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_val_ds = val_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_val_ds = preprocessed_val_ds.prefetch(tf.data.AUTOTUNE)

print(preprocessed_train_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 27), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>


## 4. モデルを作成

In [15]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

# 予測に使用したいデータに応じて変更する
# チューニングしたいハイパーパラメーターを変更したい場合は変更する
def create_model():
    model = tf.keras.Sequential()

    # 入力層
    model.add(tf.keras.Input(tensor=encoded_features))

    # 隠れ層
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1)) # ドロップアウト率

    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1)) # ドロップアウト率

    # 出力層
    model.add(tf.keras.layers.Dense(4, activation='softmax'))

    # 最適化アルゴリズム、損失関数、評価関数を指定してコンパイル
    model.compile(
        # 最適化アルゴリズム
        optimizer=tf.keras.optimizers.Adam(),
        # 損失関数
        # 二値分類→binary_crossentropy、多クラス単一ラベル分類→categorical_crossentropy
        # 多クラス多ラベル分類→binary_crossentropy、回帰問題（任意の値）→mse、回帰問題（０～１の値）→mse / binary_crossentropy
        loss="categorical_crossentropy",
        # 評価関数
        metrics=["categorical_accuracy"],
    )

    return model

model = create_model()

## 5. モデルをトレーニング

In [16]:
# モデルのトレーニング
model.fit(preprocessed_train_ds, epochs=20)

Epoch 1/20
25/25 [==============================] - 0s 1ms/step - loss: 1.1648 - categorical_accuracy: 0.4412
Epoch 2/20
25/25 [==============================] - 0s 1ms/step - loss: 0.8063 - categorical_accuracy: 0.6687
Epoch 3/20
25/25 [==============================] - 0s 1ms/step - loss: 0.6631 - categorical_accuracy: 0.7613
Epoch 4/20
25/25 [==============================] - 0s 1ms/step - loss: 0.5459 - categorical_accuracy: 0.8175
Epoch 5/20
25/25 [==============================] - 0s 4ms/step - loss: 0.4597 - categorical_accuracy: 0.8625
Epoch 6/20
25/25 [==============================] - 0s 1ms/step - loss: 0.4144 - categorical_accuracy: 0.8712
Epoch 7/20
25/25 [==============================] - 0s 1ms/step - loss: 0.3719 - categorical_accuracy: 0.8863
Epoch 8/20
25/25 [==============================] - 0s 1ms/step - loss: 0.3380 - categorical_accuracy: 0.8763
Epoch 9/20
25/25 [==============================] - 0s 1ms/step - loss: 0.3190 - categorical_accuracy: 0.8950
Epoch 10/2

## 6. モデルの評価

In [17]:
results = model.evaluate(preprocessed_val_ds, verbose=1)
print(results)

7/7 [==============================] - 0s 6ms/step - loss: 0.4179 - categorical_accuracy: 0.8700
[0.4179413616657257, 0.8700000047683716]


## 7. モデルの保存と概要

In [18]:
# モデルの保存
model.save("ranking_category_model")
# モデルの概要
model.summary()

INFO:tensorflow:Assets written to: ranking_category_model\assets


INFO:tensorflow:Assets written to: ranking_category_model\assets


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                1792      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 4004 (15.64 KB)
Trainable params: 4004 (15.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
